#### Aplicación de índices espaciales a datos geoespaciales locales
* Objeto base: unión entre barrios y radios censales (shapefile). Datos oficiales actuales.
* Atributos relevantes: radios etiquetados con cantidad de población y viviendas.
* Herramientas: ```jupyter notebook```, ```python```, ```pandas```, ```geopandas```, ```import matplotlib.pyplot```, ```folium```, ```h3```.

Algunas funciones de h3 están desactualizadas.